# AI Bus Trip Planner – ADK Capstone Project

This project demonstrates an AI-powered bus trip planner agent, created for the Kaggle Capstone Project using the Agent Development Kit (ADK). The agent helps users plan bus journeys from home to college (and back), with features like interactive memory, custom trip scheduling, and dynamic tool invocation.

**Key capabilities:**
- Remembers your home, college, and preferred departure time (via memory tool).
- Recommends the best bus based on your request and the latest timetable.
- Allows both direct and natural language queries.

---

In [ ]:
import logging

logging.getLogger("google_genai.types").setLevel(logging.ERROR)


In [ ]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="Warning: there are non-text parts in the response"
)

def chat_message(role, text):
    print(f"{role}: {text}\n")

def extract_text(resp):
    try:
        parts = resp.candidates[0].content.parts
        return "".join([p.text for p in parts if hasattr(p, 'text')])
    except:
        return str(resp)


In [ ]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("ADK components imported successfully.")

In [ ]:
import os
from kaggle_secrets import UserSecretsClient
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print(" Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

In [ ]:
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  
    exp_base=7,  
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  
)

In [ ]:
# A small in-notebook bus timetable for testing
bus_timetable = [
    {"route": "Home to College", "bus_no": "101", "departure": "08:15", "arrival": "08:55"},
    {"route": "Home to College", "bus_no": "102", "departure": "08:45", "arrival": "09:25"},
    {"route": "College to Home", "bus_no": "201", "departure": "16:10", "arrival": "16:50"},
    {"route": "College to Home", "bus_no": "202", "departure": "17:00", "arrival": "17:40"},
]

***Tool Fuction***
---
---

In [ ]:
def get_bus_options(from_stop: str, to_stop: str, after_time: str) -> dict:
    options = []
    for entry in bus_timetable:
        if entry["route"] == f"{from_stop} to {to_stop}" and entry["departure"] >= after_time:
            options.append(entry)
    return {"status": "success", "options": options}

def estimate_travel_time(from_stop: str, to_stop: str) -> dict:
    if {from_stop, to_stop} == {"Home", "College"}:
        return {"status": "success", "minutes": 40}
    return {"status": "success", "minutes": 30}

user_profile = {"home": None, "college": None, "default_departure": None}

def set_user_profile(home: str, college: str, default_departure: str) -> dict:
    user_profile["home"] = home
    user_profile["college"] = college
    user_profile["default_departure"] = default_departure
    return {"status": "success", "profile": user_profile}

def get_user_profile() -> dict:
    return {"status": "success", "profile": user_profile}

In [ ]:
bus_agent = LlmAgent(
    name="bus_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a smart bus trip planner.
    Use set_user_profile() and get_user_profile() to store and recall the user's default home/college stops and preferred departure time.
    Use get_bus_options() and estimate_travel_time() as before.
    """,
    tools=[get_bus_options, estimate_travel_time, set_user_profile, get_user_profile],
)

In [ ]:
bus_runner = InMemoryRunner(agent=bus_agent)

#### Setting User Profile Example

---

In [ ]:
response = await bus_runner.run_debug(
    "My home is Home, my college is College, and I usually leave at 08:30."
)

if isinstance(response, list):
    agent_reply = response[-1]
    print(agent_reply)
else:
    agent_reply = extract_text(response)
chat_message("Agent", agent_reply)

### Example Conversations

```
# Set user profile (memory)
response = await bus_runner.run_debug(
    "My home is Home, my college is College, and I usually leave at 08:30."
)
print(response[-1])

# Plan a trip using your saved preferences
response = await bus_runner.run_debug(
    "Plan my usual morning trip."
)
print(response[-1])
```

The agent will reply with a bus recommendation using its internal tools and memory.
```

In [ ]:
response = await bus_runner.run_debug(
    "Plan my usual morning trip."
)
agent_reply = extract_text(response)
chat_message("Agent", agent_reply)



#### > Plan Morning Trip Using Memory
*Agent reply: “I've noted that your home stop is Home, your college is College, and your default departure time is 08:30.”*

In [ ]:
response = await bus_runner.run_debug(
    "I want to go from Home to College after 08:45."
)
agent_reply = extract_text(response)
chat_message("Agent", agent_reply)


#### Plan Custom Trip
*Agent reply: “The next bus (No. 102) from Home to College departs at 08:15, arrives at 08:55. Travel time: 40 minutes.”*

---

***Implementation***

*This project uses Google’s Agent Development Kit (ADK) with a Gemini model to build a function-calling agent that can read a small in-notebook bus timetable and call custom Python tools. Four core tools are implemented: one to fetch bus options, one to estimate travel time, and two to store and retrieve the user profile (home, college, default departure time). An LlmAgent is configured with instructions to use these tools, and an InMemoryRunner manages the conversation flow so that natural language prompts are transformed into tool calls and final responses.*

***Why this project***

*Many students and commuters rely on fixed bus routes between home and college but do not have an easy way to check the best option for a specific time each day. Timetables are often static, hard to read, and require manual scanning, which is slow and error-prone during busy mornings. An AI agent that understands natural language and remembers personal preferences can make this daily planning faster, more accurate, and more accessible for all age groups.*

***Benefits***

Personalized experience: The agent remembers the user’s home, college, and preferred departure time, so repeated queries become shorter and more natural.

Faster decisions: Instead of scanning tables, users simply ask questions like “Plan my usual morning trip,” and immediately see the recommended bus.

Extensible design: The same architecture can scale to real-time bus APIs or city-wide routes, making it suitable as a starting point for a production transit assistant.



***Conclusion***

*The AI Bus Trip Planner demonstrates how an agent with memory and function tools can solve a focused, real‑world problem: daily bus trip planning between fixed locations. By remembering a user’s usual home and college stops, along with their preferred departure time, the agent reduces repetitive input and makes the experience feel more like talking to a personal assistant than working with a traditional timetable.*

*Although the current prototype relies on a small, static schedule and simple rule-based logic, its architecture is deliberately designed to be extensible. The same pattern of tools and state can be connected to live transit APIs, GPS feeds, and richer user profiles to support dynamic routes, real-time delays, and multiple cities or institutions.*

*This project therefore serves as a compact but complete example of how conversational agents can wrap complex transport data in a friendly interface. With further development such as error handling, multilingual support, and integration into mobile or kiosk interfaces—the Bus Trip Planner could evolve from a classroom prototype into a practical assistant for everyday commuters.*

---